In [1]:
import requests
import pandas as pd
import os

data_df = pd.read_csv('../data/city_bikes.csv')

foursquare_api_key = os.environ.get('FOURSQUARE_API_KEY')
yelp_api_keys = [os.environ.get('YELP_API_KEY_' + str(i)) for i in range(1, 5)]
foursquare_api_key, yelp_api_keys

(None, [None, None, None, None])

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [6]:
query = "restaurants"
# lat = "45.617499783128075"
# long = "-73.60601127147675"
radius = "1000"
fields = "name,geocodes,location,categories,distance,rating"

In [7]:
def get_foursquare_df(lat, long, name):
    url = "https://api.foursquare.com/v3/places/search?query=" + query + "&ll=" + str(lat) + "%2C" + str(long) + "&radius=" + radius + "&fields=" + fields
    # url = "https://api.foursquare.com/v3/places/search?query=" + query + "&ll=" + lat + "%2C" + long + "&radius=" + radius

    f_headers = {
        "accept": "application/json",
        "Authorization": foursquare_api_key
    }

    f_response = requests.get(url, headers=f_headers).json()
    if not f_response.get('results'):
        print(lat, long, "No results found")
    foursquare_df = pd.json_normalize(f_response.get('results'))
    foursquare_df['station_lat'] = lat
    foursquare_df['station_long'] = long
    foursquare_df['station_name'] = name
    return foursquare_df

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [83]:
for row in data_df.head(10).itertuples():
    print(row)

Pandas(Index=0, empty_slots=10, free_bikes=0, id='72bfd647b3d2b650546f42319729757d', latitude=45.61749978312808, longitude=-73.60601127147675, name='Cégep Marie-Victorin', timestamp='2023-07-28T20:56:21.092000Z', _8=0, _9=True, _10=1690577762, _11="['creditcard', 'key']", _12=True, _13=1, _14=1, _15=11, _16=693)
Pandas(Index=1, empty_slots=2, free_bikes=8, id='36c6491aa1b52e5ef7005f984738de27', latitude=45.516926210319546, longitude=-73.56425732374191, name="Gare d'autocars de Montréal (Berri / Ontario)", timestamp='2023-07-28T20:56:20.849000Z', _8=1, _9=True, _10=1690577758, _11="['creditcard', 'key']", _12=True, _13=1, _14=1, _15=15, _16=740)
Pandas(Index=2, empty_slots=14, free_bikes=3, id='8db822a266b5ccb3a1e323ddc8721d62', latitude=45.506175841460966, longitude=-73.71118605136871, name='Ateliers municipaux de St-Laurent (Cavendish / Poirier)', timestamp='2023-07-28T20:56:21.249000Z', _8=1, _9=True, _10=1690577643, _11="['creditcard', 'key']", _12=True, _13=1, _14=1, _15=18, _16=76

Put your parsed results into a DataFrame

In [8]:
foursquare_frames = [get_foursquare_df(row['latitude'], row['longitude'], row['name']) for _, row in data_df.iterrows()]
foursquare_df = pd.concat(foursquare_frames)
foursquare_df

45.43914465255811 -73.59173655509949 No results found
45.43656827071224 -73.70836973190308 No results found
45.64954588445726 -73.49191546440125 No results found
45.43791516514076 -73.58253550531117 No results found
45.61316323215739 -73.50989364087582 No results found


,categories,distance,name,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,...,location.region,rating,station_lat,station_long,station_name,geocodes.drop_off.latitude,geocodes.drop_off.longitude,location.address_extended,geocodes.front_door.latitude,geocodes.front_door.longitude
0,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",240.0,Resto Capucine,45.619509,-73.609503,45.619509,-73.609503,7000 Maurice du Plessis Blvd,CA,Coin Albert-Hudon,...,QC,NaN,45.61750,-73.606011,Cégep Marie-Victorin,NaN,NaN,NaN,NaN,NaN
1,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",587.0,Biscuits Heavenly Taste Biscotti,45.622126,-73.601188,45.622126,-73.601188,11737 4e Ave,CA,,...,QC,NaN,45.61750,-73.606011,Cégep Marie-Victorin,NaN,NaN,NaN,NaN,NaN
2,"[{'id': 13236, 'name': 'Italian Restaurant', '...",941.0,Restaurant Prima Luna,45.617439,-73.593995,45.617439,-73.593995,7301 Henri-Bourassa Blvd E,CA,,...,QC,7.4,45.61750,-73.606011,Cégep Marie-Victorin,NaN,NaN,NaN,NaN,NaN
3,"[{'id': 13352, 'name': 'Thai Restaurant', 'ico...",696.0,Restaurant Phoenix Oriental,45.623300,-73.602149,45.623300,-73.602149,7230 Maurice-Duplessis Blvd,CA,Maurice Duplessis,...,QC,NaN,45.61750,-73.606011,Cégep Marie-Victorin,NaN,NaN,NaN,NaN,NaN
4,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",773.0,Restaurant Shekz,45.623543,-73.601380,45.623543,-73.601380,7272 Maurice-Duplessis Blvd,CA,NaN,...,QC,NaN,45.61750,-73.606011,Cégep Marie-Victorin,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,"[{'id': 13027, 'name': 'Bistro', 'icon': {'pre...",930.0,Restaurant Chez la Mère,45.558488,-73.568612,45.558488,-73.568612,4028 Masson Rue,CA,Btw Jean-d'Arc & Pie-IX,...,QC,7.4,45.56664,-73.571596,Parc de la Louisiane (31e avenue / de Bellecha...,45.558583,-73.568758,NaN,NaN,NaN
6,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",851.0,Beaubien Deli,45.562001,-73.580362,45.562001,-73.580362,4023 Beaubien Rue E,CA,NaN,...,QC,6.7,45.56664,-73.571596,Parc de la Louisiane (31e avenue / de Bellecha...,NaN,NaN,NaN,NaN,NaN
7,"[{'id': 13276, 'name': 'Sushi Restaurant', 'ic...",693.0,Hokkado,45.563501,-73.579246,45.563501,-73.579246,4205 Beaubien Rue E,CA,,...,QC,NaN,45.56664,-73.571596,Parc de la Louisiane (31e avenue / de Bellecha...,45.563338,-73.579055,NaN,NaN,NaN
8,"[{'id': 13065, 'name': 'Restaurant', 'icon': {...",738.0,La Grotta del Mare,45.562981,-73.579666,45.562981,-73.579666,4111 Beaubien Rue E,CA,,...,QC,NaN,45.56664,-73.571596,Parc de la Louisiane (31e avenue / de Bellecha...,NaN,NaN,NaN,NaN,NaN


In [9]:
foursquare_df.keys()

Index(['categories', 'distance', 'name', 'geocodes.main.latitude',
       'geocodes.main.longitude', 'geocodes.roof.latitude',
       'geocodes.roof.longitude', 'location.address', 'location.country',
       'location.cross_street', 'location.formatted_address',
       'location.locality', 'location.postcode', 'location.region', 'rating',
       'station_lat', 'station_long', 'station_name',
       'geocodes.drop_off.latitude', 'geocodes.drop_off.longitude',
       'location.address_extended', 'geocodes.front_door.latitude',
       'geocodes.front_door.longitude'],
      dtype='object')

In [10]:
foursquare_df.to_csv('foursquare_df.csv', index=False)

In [45]:
total = foursquare_df.isnull().sum().sort_values(ascending=False)
total.head(20)

geocodes.front_door.longitude    7443
geocodes.front_door.latitude     7443
location.address_extended        7053
geocodes.drop_off.longitude      3773
geocodes.drop_off.latitude       3773
rating                           1641
location.cross_street             617
location.postcode                 145
geocodes.roof.latitude             39
geocodes.roof.longitude            39
location.address                   29
location.region                     1
location.locality                   1
categories                          0
distance                            0
location.country                    0
geocodes.main.longitude             0
geocodes.main.latitude              0
name                                0
location.formatted_address          0
dtype: int64

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [62]:
lat = "45.6175"
long = "-73.606011"
term = "restaurants"

In [67]:
def get_yelp_df(lat, long, name, token):
    url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(lat) + "&longitude=" + str(long) + "&term=" + term + "&radius=" + radius

    yelp_headers = {
        "accept": "application/json",
        "Authorization": "Bearer " + token
    }

    yelp_response = requests.get(url, headers=yelp_headers).json()
    if len(yelp_response.get('businesses')) == 0:
        print(lat, long, name, "No results from Yelp API")
    yelp_df = pd.json_normalize(yelp_response.get('businesses'))
    yelp_df['station_lat'] = lat
    yelp_df['station_long'] = long
    yelp_df['station_name'] = name
    return yelp_df


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [64]:
len(data_df)

789

In [77]:
yelp_frames = []

for i in range(0, len(data_df)):
    yelp_frames.append(get_yelp_df(data_df.loc[i, 'latitude'], data_df.loc[i, 'longitude'], data_df.loc[i, 'name'], yelp_api_keys[i // 200]))
yelp_df = pd.concat(yelp_frames)
yelp_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address,station_lat,station_long,station_name
0,BpnevNTdiTqQpLD21cRg1A,grillades-sizzle-montréal,Grillades Sizzle,https://s3-media1.fl.yelpcdn.com/bphoto/Pc-ERL...,False,https://www.yelp.com/biz/grillades-sizzle-mont...,3.0,"[{'alias': 'portuguese', 'title': 'Portuguese'}]",3.5,[],...,NaN,NaN,Montreal,H1E 1M4,CA,QC,"['7300, boul Maurice-Duplessis', 'Montreal, QC...",45.6175,-73.606011,Cégep Marie-Victorin
1,vRJPrk-VH75uxLiVVprBhA,capucine-montreal-nord,Capucine,https://s3-media2.fl.yelpcdn.com/bphoto/BOs9f6...,False,https://www.yelp.com/biz/capucine-montreal-nor...,4.0,"[{'alias': 'italian', 'title': 'Italian'}]",5.0,[],...,NaN,NaN,Montreal-Nord,H1G,CA,QC,"['7000 Boulevard Maurice Duplessis', 'Montreal...",45.6175,-73.606011,Cégep Marie-Victorin
2,qXByAmujOzT_9lm1_KGsiw,shekz-restaurant-montréal,Shekz Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/9yafDh...,False,https://www.yelp.com/biz/shekz-restaurant-mont...,2.0,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",1.5,[],...,NaN,NaN,Montreal,H1E 6Z7,CA,QC,"['7272 Boulevard Maurice-Duplessis', 'Montreal...",45.6175,-73.606011,Cégep Marie-Victorin
3,TyKGLi1V1PFRHYmHDjFUMQ,restaurant-prima-luna-montréal-2,Restaurant Prima Luna,https://s3-media2.fl.yelpcdn.com/bphoto/MW1rfW...,False,https://www.yelp.com/biz/restaurant-prima-luna...,14.0,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,[],...,NaN,NaN,Montreal,H1E 2Z6,CA,QC,"['7301Boulevard Henri-Bourassa E', 'Montreal, ...",45.6175,-73.606011,Cégep Marie-Victorin
4,2P1Rvd4IboY8z7oRoT8b8g,dagostino-pizza-montréal-2,Dagostino Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/GxNsOS...,False,https://www.yelp.com/biz/dagostino-pizza-montr...,3.0,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],...,NaN,NaN,Montreal,H1E 3Y1,CA,QC,"['7470 Maurice-Duplessis Boul', 'Montreal, QC ...",45.6175,-73.606011,Cégep Marie-Victorin


In [76]:
yelp_df.to_csv('yelp_df.csv', index=False)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In [26]:
foursquare_df = pd.read_csv('foursquare_df.csv')
# print(foursquare_df['name'].unique().shape)
print(foursquare_df.groupby('station_name').count().mean()['name'])

yelp_df = pd.read_csv('yelp_df.csv')
# print(yelp_df['name'].unique().shape)
print(yelp_df.groupby('station_name').count().mean()['name'])

9.542091836734693
19.271573604060915


While Foursquare gets an average of 9.5 locations per bike station, Yelp gets an average of 19.2 points of interest, which is more than double

Get the top 10 restaurants according to their rating

In [42]:
yelp_df = pd.read_csv('../data/yelp_df.csv')
yelp_df['rating'] = yelp_df['rating'].apply(lambda x: x*2)
yelp_df = yelp_df.sort_values('rating', ascending=False)
# yelp_df.head()
foursquare_df = pd.read_csv('../data/foursquare_df.csv')
total_df = pd.concat([yelp_df, foursquare_df], axis=0).reset_index(drop=True)
print(len(total_df))
total_df = total_df.drop_duplicates(subset=['name'], keep='first')
print(len(total_df))

total_df.to_csv('../data/total_df.csv', index=False)
# total_df = total_df.sort_values('rating', ascending=False)
# total_df.head(10)[['name', 'rating']].reset_index(drop=True)

22667
3805
